In [1]:
import pandas as pd
import json
import pandas as pd
import requests
import re
import numpy as np
from time import sleep

def read_all_sheets(fname='input/locations.xlsx'):
    all_data = pd.read_excel(fname, sheet_name=None)
    for continent in all_data:
        all_data[continent]['continent'] = continent
    return pd.concat(all_data.values())


def parse_page(url: str):
    snowpack = {}
    measurement_date = {}

    page = requests.get(url)
    lines = (l.decode("utf-8") for l in page.iter_lines())
    snowfalls_pattern = re.compile('jssnowfalls([0-9]+)\s*=\s*(\[.*\])')
    dates_pattern = re.compile('jsdates([0-9]+)\s*=\s*(\[.*\])')
    
    for line in lines:
        snowfalls_match = snowfalls_pattern.findall(line)
        dates_match = dates_pattern.findall(line)
        if snowfalls_match:
            assert len(snowfalls_match)==1
            text_match = snowfalls_match[0]
            year = int(text_match[0])
            data = json.loads(text_match[1])
            snowpack[year] = data
        if dates_match:
            assert len(dates_match)==1
            text_match = dates_match[0]
            year = int(text_match[0])
            data = json.loads(text_match[1])
            measurement_date[year] = data
    return snowpack, measurement_date
    

def dicts_to_df(snow_data, dates, data_type):
    yearly_dfs = []
    for year in snow_data.keys():
        df = pd.DataFrame(zip(snow_data[year], dates[year]),
                          columns=[data_type, 'date'])
        df['season'] = year
        yearly_dfs.append(df)
    out = pd.concat(yearly_dfs)
    return out

def get_one_resort(continent: str, region: str, resort: str):
    
    resort_base_url = "/".join(["https://www.onthesnow.com", region, resort, "historical-snowfall.html?y=0"]) # y=0 gets multiple years in one call
    snowpack_url = resort_base_url + "&q=top"
    snowfall_url = resort_base_url + "&q=snow"    
    snowpack, snowpack_dates = parse_page(snowpack_url)
    snowpack_df = dicts_to_df(snowpack, snowpack_dates, "snowpack")
    snowpack_df = snowpack_df[snowpack_df.snowpack>0]
    snowpack_df = snowpack_df[snowpack_df.snowpack<400]
    snowfall, snowfall_dates = parse_page(snowfall_url)
    snowfall_df = dicts_to_df(snowfall, snowfall_dates, "snowfall")
    for df in (snowpack_df, snowfall_df):
        df['continent'] = continent
        df['region'] = region
        df['resort'] = resort
        df['date'] = pd.to_datetime(df.date)
        df['day_of_year'] = df.date.dt.dayofyear
    return {'snowpack': snowpack_df, 'snowfall': snowfall_df}

def stochastic_sleep(min, max):
    sleep(np.random.uniform(min, max))

In [2]:
snow_data_accumulator = []

locations = read_all_sheets()

for index, row_data in locations.iterrows():
    # one source of erorrs is getting the resort name wrong do to character encoding stuff
    try:
        snow_data_accumulator.append(get_one_resort(row_data.continent, row_data.Region, row_data.Resort))
    except:
        print(f"Error on {row_data.Resort}")
    stochastic_sleep(1, 2)

snowpack = pd.concat(resort_data['snowpack'] for resort_data in snow_data_accumulator)
snowfall = pd.concat(resort_data['snowfall'] for resort_data in snow_data_accumulator)

Error on bjoerkliden
Error on meribe
Error on riksgransen
Error on crystal-mountain-wa
Error on silverton-mountain


In [3]:
snowpack.to_csv('output/snowpack.csv', index=False)
snowfall.to_csv('output/snowfall.csv', index=False)

In [4]:
snowpack = pd.read_csv('output/snowpack.csv', parse_dates=['date'])
snowfall = pd.read_csv('output/snowfall.csv', parse_dates=['date'])

snowpack['dayofyear'] = snowpack.date.dt.dayofyear
snowfall['dayofyear'] = snowfall.date.dt.dayofyear

DAY_TO_TRACK = 92
usable_seasons = snowpack.query('season > 2011 and season < 2019')
# Some resorts don't report every day... so take the first one after ...
days_after_threshold = usable_seasons[usable_seasons.date.dt.dayofyear>DAY_TO_TRACK]
to_plot = days_after_threshold.loc[days_after_threshold.groupby(["resort", "season"])["dayofyear"].idxmin()]

In [7]:
from IPython.display import display
import altair as alt

for resort in to_plot.resort.unique():
    current_resort_data = to_plot[to_plot.resort == resort]
    snowpack_chart = alt.Chart(current_resort_data, title=resort).mark_point().encode(
        alt.X('season:N'),
        alt.Y('snowpack', title='Snowpack on April 1')
    ).properties(width=600)
    
#     display(snowpack_chart + snowpack_chart.transform_regression('season', 'snowpack').mark_line())

In [8]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

exog = sm.add_constant(to_plot.season)
model = smf.OLS(endog=to_plot.snowpack, exog = exog, data=to_plot).fit()
model.summary()

/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               snowpack   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.3289
Date:                Sat, 05 Dec 2020   Prob (F-statistic):              0.566
Time:                        19:27:32   Log-Likelihood:                -20386.
No. Observations:                3932   AIC:                         4.078e+04
Df Residuals:                    3930   BIC:                         4.079e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -343.0023    690.437     -0.497      0.619   -1696.651    1010.646
season         0.1965      0.343      0.574      0.566      -0.475       0.868
==============================================================================
Omnibus:                     1125.144   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2966.936
Skew:                           1.536   Prob(JB):                         0.00
Kurtosis:                       5.945   Cond. No.                     2.02e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.02e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
total_snowfall

,continent,season,snowfall
0,Europe,2012,27585
1,Europe,2013,25851
2,Europe,2014,23935
3,Europe,2015,22796
4,Europe,2016,18907
5,Europe,2017,34713
6,Europe,2018,29860
7,North America,2012,43636
8,North America,2013,48061
9,North America,2014,33581


In [26]:
snowfall.query('season > 2011 and season < 2019').groupby(['continent', 'season']).resort.nunique()

continent      season
Europe         2012      214
               2013      216
               2014      217
               2015      217
               2016      217
               2017      217
               2018      217
North America  2012      357
               2013      361
               2014      362
               2015      362
               2016      361
               2017      360
               2018      361
South America  2012       11
               2013       11
               2014       11
               2015       10
               2016       11
               2017       11
               2018       11
Name: resort, dtype: int64

In [22]:
total_snowfall = snowfall.query('season > 2011 and season < 2019').groupby(['continent', 'season']).snowfall.sum().reset_index()

alt.Chart(total_snowfall).mark_point().encode(
    alt.X('season:N'),
    alt.Y('snowfall'),
    alt.Color('continent')
)

alt.Chart(...)